# Heisenberg model with MPS DMRG

- fare versione self-developed e confronto con preconfezionata
- con la custom variare per esempio bond dim
- calcolare magari magnetization e correlation distance (medie)
- perché avere una simmetria permette di ottimizzare? (mi sa che ce ne freghiamo in ITensors)
- a quanto pare l'inizializzazione inziale conta (effettivamente, se ci sono due stati degeneri...)
- dato che si lavora con lo spin in 3 dim misurare tutte
- approccio mps funziona per via della natura locale dell'hamiltoniana? e quindi poca correlazione quantistica e buona compressione? (perché in linea di principio facendo un idmrg classico si ha la compressione nel passaggio a N più grande, ma si continua a diagonalizzare in maniera esatta e non compressa come con lo sweep in questo caso)
- tensor svd: si possono sempre riarrangiare gli indici liberi in uno unico, in questo modo si ottengono un indice libero e uno contratto con s. Chiaramente nel tensore iniziale c'è arbitrarietà su come raggruppare gli indici liberi e di conseguenza che prodotto finale ottenere

In [ ]:
using ITensors
using PyPlot


i = Index(2,"i")
A = randomITensor(i',i)

@show A

In [ ]:
A[i=>1,i'=>2] = 1
@show A

## Off the shelf

In [ ]:
using ITensors
using PyPlot

In [ ]:
function HXXZ_dmrg_ready(N, Jxx, Jz)
    
  sites = siteinds("S=1/2",N)

  ampo = OpSum()
  for j=1:N-1
    ampo += Jxx*0.5,"S+",j,"S-",j+1
    ampo += Jxx*0.5,"S-",j,"S+",j+1
    ampo += Jz,"Sz",j,"Sz",j+1
  end
  H = MPO(ampo,sites)

  sweeps = Sweeps(5) # number of sweeps is 5
  maxdim!(sweeps,10,20,100,100,200) # gradually increase states kept
  cutoff!(sweeps,1E-10) # desired truncation error

  psi0 = randomMPS(sites,2)

  energy,psi = dmrg(H,psi0,sweeps)

  return energy, psi
end

In [ ]:
trial = []

for n in [a^2 for a in 5:20:100]
    E, psi = HXXZ_dmrg_ready(n,0.2,0.5)
    push!(trial, E/n)
end

In [ ]:
trial

In [ ]:
N = 100
Jxx = 1.0
Deltas = [0.1,0.3,0.8,1.5,2.0,3.0]

e = []
gr_st = []

for delta in Deltas
    E, psi = HXXZ_dmrg_ready(N,Jxx,delta*Jxx)
    push!(e, E/N)
    push!(gr_st, psi)
end


In [ ]:
plot(Deltas, e);

In [ ]:
zzcorr = []
for i in gr_st
    push!(zzcorr, correlation_matrix(i,"Sz","Sz"))
end

In [ ]:
zcor_m = []

for i in zzcorr
    mean = 0.
    count = 0
    for j in 1:length(i[:,1])
        for k in j+1:length(i[1,:])
            mean += i[j,k]
            count += 1
        end
    end
    mean = mean/count
    push!(zcor_m, mean)
end
        

In [ ]:
plot(Deltas, zcor_m);

## Self-made

In [1]:
using ITensors
using PyPlot

In [7]:
function HXXZ_idmrg(N, Jxx, Jz, sweeps=3, bond=4, max_err=1e-2)
    
    # N must be even
    if iseven(N) == false
        N -= 1
    end
    if N > 10
        print("error: unit cell size N must be <= 10")
    end
    if max_err >= 1
        print("error: max_err must be < 1")
    end
    
    o_ind = [Index(2) for i=1:N]
    bonds = [Index(bond) for i=1:N-1]
    
    Sz = ITensor[]
    Sp = ITensor[]
    Sm = ITensor[]
    Id = ITensor[]
    psi = ITensor[]
    psi_bar = ITensor[]
    for i=1:N
        push!(Sz, ITensor(o_ind[i], o_ind[i]'))  #forgetting factor 1/2 here and in the Sx/Sy
        Sz[i][o_ind[i]=>1, o_ind[i]'=>1] = 1
        Sz[i][o_ind[i]=>1, o_ind[i]'=>2] = 0
        Sz[i][o_ind[i]=>2, o_ind[i]'=>1] = 0
        Sz[i][o_ind[i]=>2, o_ind[i]'=>2] = -1
        
        push!(Sp, ITensor(o_ind[i], o_ind[i]'))
        Sp[i][o_ind[i]=>1, o_ind[i]'=>1] = 0
        Sp[i][o_ind[i]=>1, o_ind[i]'=>2] = 1
        Sp[i][o_ind[i]=>2, o_ind[i]'=>1] = 0
        Sp[i][o_ind[i]=>2, o_ind[i]'=>2] = 0
        
        push!(Sm, ITensor(o_ind[i], o_ind[i]'))
        Sm[i][o_ind[i]=>1, o_ind[i]'=>1] = 0
        Sm[i][o_ind[i]=>1, o_ind[i]'=>2] = 1
        Sm[i][o_ind[i]=>2, o_ind[i]'=>1] = 0
        Sm[i][o_ind[i]=>2, o_ind[i]'=>2] = 0
        
        push!(Id, ITensor(o_ind[i], o_ind[i]'))
        Id[i][o_ind[i]=>1, o_ind[i]'=>1] = 1
        Id[i][o_ind[i]=>1, o_ind[i]'=>2] = 0
        Id[i][o_ind[i]=>2, o_ind[i]'=>1] = 0
        Id[i][o_ind[i]=>2, o_ind[i]'=>2] = 1
        
        if i == 1
            push!(psi, randomITensor(o_ind[i], bonds[i]))
            push!(psi_bar, prime(psi[i]))
        elseif i == N
            push!(psi, randomITensor(o_ind[i], bonds[i-1]))
            push!(psi_bar, prime(psi[i]))
        else
            push!(psi, randomITensor(o_ind[i], bonds[i-1], bonds[i]))
            push!(psi_bar, prime(psi[i]))
        end
    end
    
    norm1 = psi[1]
    for i=2:N
        norm1 = norm1 * psi[i]
    end
    norm1 = norm(norm1)
    psi = psi/sqrt(norm1)
    psi_bar = psi_bar/sqrt(norm1)
    
    H = ITensor()
    
    for i=1:N-1
        temp = Jxx*(Sp[i]*Sm[i+1]+Sm[i]*Sp[i+1]) + Jz*Sz[i]Sz[i+1]
        for j=1:i-1
            temp = temp * Id[j]
        end
        for j=i+2:N
            temp = temp * Id[j]
        end
        H += temp
    end
    H = H #*0.5
        
    """
    SWEEP
    """
    for s=1:sweeps
        for i=2:N
            block = ITensor()
            for j=1:N
                if j == 1
                    block = H
                end
                block = block * psi_bar[j]
                if (j<(i-1)) | (j>i)
                    block = block * psi[j]
                end
            end

            if i == 2
                U,S,V = svd(block, o_ind[1]; maxdim=bond)
                psi[1] = U
                bonds[1] = commoninds(U,S)[1]
                psi_bar[1] = prime(psi[1])
                psi[2] = S*V
                psi_bar[2] = prime(psi[2])
            else
                U,S,V = svd(block, o_ind[i-1], bonds[i-2]; maxdim=bond)
                psi[i-1] = U
                bonds[i-1] = commoninds(U,S)[1]
                psi_bar[i-1] = prime(psi[i-1])
                psi[i] = S*V
                psi_bar[i] = prime(psi[i])
            end
        end

        ### NORM STATE ###
        #norm = psi[1]*noprime(psi_bar[1])
        #for i=2:N
        #    norm = norm*psi[i]*noprime(psi_bar[i])
        #end
        #psi = psi/sqrt(norm[1])
        #psi_bar = psi_bar/sqrt(norm[1])
        
        norm1 = psi[1]
        for i=2:N
            norm1 = norm1 * psi[i]
        end
        norm1 = norm(norm1)
        psi = psi/sqrt(norm1)
        psi_bar = psi_bar/sqrt(norm1)
        
        energy = H
        for i=1:N
            energy = energy * psi_bar[i] * psi[i]
        end
        print(energy)

        #for i=N:-1:2
        #    block = ITensor()
        #    for j=1:N
        #        if j == 1
        #            block = H
        #        end
        #        block = block * psi_bar[j]
        #        if (j<(i-1)) | (j>i)
        #            block = block * psi[j]
        #        end
        #    end
        #
        #    if i == 2
        #        U,S,V = svd(block, o_ind[1]; maxdim=bond)
        #        psi[1] = U
        #        bonds[1] = commoninds(U,S)[1]
        #        psi_bar[1] = prime(psi[1])
        #        psi[2] = S*V
        #        psi_bar[2] = prime(psi[2])
        #    else
        #        U,S,V = svd(block, o_ind[i-1], bonds[i-2]; maxdim=bond)
        #        psi[i-1] = U
        #        bonds[i-1] = commoninds(U,S)[1]
        #        psi_bar[i-1] = prime(psi[i-1])
        #        psi[i] = S*V
        #        psi_bar[i] = prime(psi[i])
        #    end
        #end
    end
    
    
    H_l = ITensor()
    H_r = ITensor()
    for i=1:Integer(N/2)-1
        temp = Jxx*(Sp[i]*Sm[i+1]+Sm[i]*Sp[i+1]) + Jz*Sz[i]Sz[i+1]
        for j=1:i-1
            temp = temp * Id[j]
        end
        for j=i+2:Integer(N/2)
            temp = temp * Id[j]
        end
        H_l += temp
    end
    for i=Integer(N/2)+1:N-1
        temp = Jxx*(Sp[i]*Sm[i+1]+Sm[i]*Sp[i+1]) + Jz*Sz[i]Sz[i+1]
        for j=Integer(N/2)+1:i-1
            temp = temp * Id[j]
        end
        for j=i+2:N
            temp = temp * Id[j]
        end
        H_r += temp
    end
    
    err = 1
    prev = 0
    left = ITensor()
    right = ITensor()
    energy = ITensor()
    first = 0
    eff_dim = N
    push!(bonds, Index(bond))
    push!(bonds, Index(bond))
    while first == 0 #err > max_err
        
        """
        UNIT CELL ADDITION
        """
        eff_dim = eff_dim + N
        if first == 0
            first += 1
            left = psi[1] * H_l * psi_bar[1]
            right = psi[N] * H_r * psi_bar[N]
        else
            left = left * psi[1] * H_l * psi_bar[1]
            right = right * psi[N] * H_r * psi_bar[N]
        end
        #if N>2...
        for i=2:Integer(N/2)
            left = left * psi[i] * psi_bar[i]
            right = right * psi[N+1-i] * psi_bar[N+1-i]
        end
        bonds[1] = bonds[Integer(N/2)]
        bonds[N+1] = bonds[Integer(N/2)]''
        bonds[Integer(N/2)] = Index(bond)
        right = right''
        
        """
        fare un svd di left e right, sono entrambi quadrati direi, quindi dovrebbe di fatto
        uscire una decomposizione in matrici quadrate che è la trasformazione unitaria di 
        diagonalizzazione. Poi usare il U e V come sites in modo da aggiornarli anche
        """
        
        for i=1:N
            psi[i] = randomITensor(o_ind[i], bonds[i], bonds[i+1])
            psi_bar[i] = prime(psi[i])
        end
        norm1 = psi[1]
        for i=2:N
            norm1 = norm1 * psi[i]
        end
        norm1 = norm(norm1)
        psi = psi/sqrt(norm1)
        psi_bar = psi_bar/sqrt(norm1)
        
        for s=1:sweeps
            for i=2:N
                block = ITensor()
                for j=1:N
                    if j == 1
                        block = H * left * right
                    end
                    block = block * psi_bar[j]
                    if (j<(i-1)) | (j>i)
                        block = block * psi[j]
                    end
                end

                U,S,V = svd(block, o_ind[i-1], bonds[i-1]; maxdim=bond)
                psi[i-1] = U
                bonds[i-1] = commoninds(U,S)[1]
                psi_bar[i-1] = prime(psi[i-1])
                psi[i] = S*V
                psi_bar[i] = prime(psi[i])
            end
            
            norm1 = psi[1]
            for i=2:N
                norm1 = norm1 * psi[i]
            end
            norm1 = norm(norm1)
            psi = psi/sqrt(norm1)
            psi_bar = psi_bar/sqrt(norm1)

            #for i=N:-1:2
            #    block = ITensor()
            #    for j=1:N
            #        if j == 1
            #            block = H * left * right
            #        end
            #        block = block * psi_bar[j]
            #        if (j<(i-1)) | (j>i)
            #            block = block * psi[j]
            #        end
            #    end
            #
            #    U,S,V = svd(block, o_ind[i-1], bonds[i-1]; maxdim=bond)
            #    psi[i-1] = U
            #    bonds[i-1] = commoninds(U,S)[1]
            #    psi_bar[i-1] = prime(psi[i-1])
            #    psi[i] = S*V
            #    psi_bar[i] = prime(psi[i])
            #end
        end
        
        energy = H * left * right / eff_dim
        for i=1:N
            energy = energy * psi_bar[i] * psi[i]
        end
        
        err = (energy[1]-prev)/energy[1]
        prev = energy[1]
        print(energy[1])
    end  

    return energy, psi[1]
end

HXXZ_idmrg (generic function with 4 methods)

In [8]:
HXXZ_idmrg(4, 2.5, 3.6, 5)

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
-1353.3981870128243ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
-1.4729295543244196e-5ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
-7.533975596812895e-6ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
-6.862431267939654e-6ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
-6.812345389065526e-6Contraction resulted in ITensor with 14 indices, which is greater than or equal to the ITensor order warning threshold 14. You can modify the threshold with macros like `@set_warn_order N`, `@reset_warn_order`, and `@disable_warn_order` or functions like `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1788
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia

Contraction resulted in ITensor with 16 indices, which is greater than or equal to the ITensor order warning threshold 14. You can modify the threshold with macros like `@set_warn_order N`, `@reset_warn_order`, and `@disable_warn_order` or functions like `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1788
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1882
  [3] *(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1870
  [4] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64, bond::Int64, max_err::Float64)
    @ Main ./In[7]:245
  [5] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64)
    @ Main ./In[7]:4
  [6] top-level scope
    @ In[8]:1
  [7] eval
    @ ./boot.jl:373 [inlined]
  [8] include_string(map

 [14] (::IJulia.var"#15#18")()
    @ IJulia ./task.jl:429
Contraction resulted in ITensor with 14 indices, which is greater than or equal to the ITensor order warning threshold 14. You can modify the threshold with macros like `@set_warn_order N`, `@reset_warn_order`, and `@disable_warn_order` or functions like `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1788
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1882
  [3] *(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1870
  [4] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64, bond::Int64, max_err::Float64)
    @ Main ./In[7]:245
  [5] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64)
    @ Main ./In[7]:4
  [6] top-level scope
    @ In[8]:1
  [7] e

Contraction resulted in ITensor with 15 indices, which is greater than or equal to the ITensor order warning threshold 14. You can modify the threshold with macros like `@set_warn_order N`, `@reset_warn_order`, and `@disable_warn_order` or functions like `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1788
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1882
  [3] *(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1870
  [4] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64, bond::Int64, max_err::Float64)
    @ Main ./In[7]:243
  [5] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64)
    @ Main ./In[7]:4
  [6] top-level scope
    @ In[8]:1
  [7] eval
    @ ./boot.jl:373 [inlined]
  [8] include_string(map

Contraction resulted in ITensor with 16 indices, which is greater than or equal to the ITensor order warning threshold 14. You can modify the threshold with macros like `@set_warn_order N`, `@reset_warn_order`, and `@disable_warn_order` or functions like `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1788
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1882
  [3] *(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1870
  [4] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64, bond::Int64, max_err::Float64)
    @ Main ./In[7]:245
  [5] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64)
    @ Main ./In[7]:4
  [6] top-level scope
    @ In[8]:1
  [7] eval
    @ ./boot.jl:373 [inlined]
  [8] include_string(map

...

Contraction resulted in ITensor with 15 indices, which is greater than or equal to the ITensor order warning threshold 14. You can modify the threshold with macros like `@set_warn_order N`, `@reset_warn_order`, and `@disable_warn_order` or functions like `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1788
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1882
  [3] *(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1870
  [4] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64, bond::Int64, max_err::Float64)
    @ Main ./In[7]:243
  [5] HXXZ_idmrg(N::Int64, Jxx::Float64, Jz::Float64, sweeps::Int64)
    @ Main ./In[7]:4
  [6] top-level scope
    @ In[8]:1
  [7] eval
    @ ./boot.jl:373 [inlined]
  [8] include_string(map

Contraction resulted in ITensor with 14 indices, which is greater than or equal to the ITensor order warning threshold 14. You can modify the threshold with macros like `@set_warn_order N`, `@reset_warn_order`, and `@disable_warn_order` or functions like `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1788
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1882
  [3] #232
    @ ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1968 [inlined]
  [4] BottomRF
    @ ./reduce.jl:81 [inlined]
  [5] afoldl
    @ ./operators.jl:614 [inlined]
  [6] _foldl_impl
    @ ./tuple.jl:277 [inlined]
  [7] foldl_impl
    @ ./reduce.jl:48 [inlined]
  [8] mapfoldl_impl
    @ ./reduce.jl:44 [inlined]
  [9] #mapfoldl#244
    @ ./reduce.jl:162 [inlined]
 [10] mapfoldl
    @ ./reduce.jl:162 [in

Contraction resulted in ITensor with 16 indices, which is greater than or equal to the ITensor order warning threshold 14. You can modify the threshold with macros like `@set_warn_order N`, `@reset_warn_order`, and `@disable_warn_order` or functions like `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1788
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1882
  [3] #232
    @ ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1968 [inlined]
  [4] BottomRF
    @ ./reduce.jl:81 [inlined]
  [5] afoldl
    @ ./operators.jl:614 [inlined]
  [6] _foldl_impl
    @ ./tuple.jl:277 [inlined]
  [7] foldl_impl
    @ ./reduce.jl:48 [inlined]
  [8] mapfoldl_impl
    @ ./reduce.jl:44 [inlined]
  [9] #mapfoldl#244
    @ ./reduce.jl:162 [inlined]
 [10] mapfoldl
    @ ./reduce.jl:162 [in

Contraction resulted in ITensor with 16 indices, which is greater than or equal to the ITensor order warning threshold 14. You can modify the threshold with macros like `@set_warn_order N`, `@reset_warn_order`, and `@disable_warn_order` or functions like `ITensors.set_warn_order(N::Int)`, `ITensors.reset_warn_order()`, and `ITensors.disable_warn_order()`.

Stacktrace:
  [1] _contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1788
  [2] contract(A::ITensor, B::ITensor)
    @ ITensors ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1882
  [3] #232
    @ ~/.julia/packages/ITensors/NlHqJ/src/itensor.jl:1968 [inlined]
  [4] BottomRF
    @ ./reduce.jl:81 [inlined]
  [5] afoldl
    @ ./operators.jl:614 [inlined]
  [6] _foldl_impl
    @ ./tuple.jl:277 [inlined]
  [7] foldl_impl
    @ ./reduce.jl:48 [inlined]
  [8] mapfoldl_impl
    @ ./reduce.jl:44 [inlined]
  [9] #mapfoldl#244
    @ ./reduce.jl:162 [inlined]
 [10] mapfoldl
    @ ./reduce.jl:162 [in

(ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
-2.835490702511229e21, ITensor ord=3
Dim 1: (dim=2|id=865)
Dim 2: (dim=4|id=945|"Link,u")
Dim 3: (dim=4|id=243|"Link,u")
NDTensors.Dense{Float64, Vector{Float64}}
 2×4×4
[:, :, 1] =
 -3.453399023796518e-14    3.392959006351347e6  …  271114.6946182539
  1.776898758762858e-10  659.4112830160703             52.69031788711186

[:, :, 2] =
   -3.881731176032352e6  8.115333388156818e-11   …  1.3518451374817375e-8
 -754.4026704470875      2.6468712406766572e-14     2.6267889073929507e-12

[:, :, 3] =
  1.3682870084414954e-8  23023.0354593174       …    3.835157242412785e6
 -1.2908818165499623e-9      4.474457051417961     745.3511678307372

[:, :, 4] =
 -2.0928310453997024e-14    1.885506840093406e6  …  -534698.6733781263
  1.0774177402517116e-10  366.4425306142069            -103.91706400783806)